In [16]:
#!pip install simple_salesforce

In [17]:
import json
from sqlalchemy import create_engine
import pandas as pd
import pymysql
pymysql.install_as_MySQLdb()

In [18]:
# import sys
# sys.path.append('../../../../')
from config import sf_username, sf_password, sf_security_token
from config import remote_db_endpoint, remote_db_port
from config import remote_db_name, remote_db_user, remote_db_pwd

In [19]:
from simple_salesforce import Salesforce
sf = Salesforce(username=sf_username, password=sf_password, security_token=sf_security_token)

In [20]:
engine = create_engine(f"mysql://{remote_db_user}:{remote_db_pwd}@{remote_db_endpoint}:{remote_db_port}/{remote_db_name}")
conn = engine.connect()

In [21]:
course_data_df = pd.read_sql("SELECT * FROM course", conn)
course_data_df.head(30)


,ID_Course,CourseCode,CourseName,CreditHours,BootCampCourse
0,1,BC-DATAVIZ,Data Visualization and Analytics,12,1
1,2,BC-WEBDEV,Full Stack Web Development,12,1
2,3,BC-UIUX,User Interface/User Experience,12,1
3,4,CIS-349,Introduction to Databases,5,0
4,5,CIS-405,Database Programming,5,0
5,6,CIS-438,Database Administration,5,0
6,7,CIS-430,Business Systems Programming I,5,0
7,8,CIS-435,Business Systems Programming II,5,0


In [22]:
course_data_df.rename(columns={
    'ID_Course':'Name',
    'CourseCode':'Course_Code__c',
    'CourseName':'Course_Name__c',
    'CreditHours':'Credit_Hours__c',
    'BootCampCourse':'Boot_Camp_Course__c',    
}, inplace=True)
course_data_df.head()

,Name,Course_Code__c,Course_Name__c,Credit_Hours__c,Boot_Camp_Course__c
0,1,BC-DATAVIZ,Data Visualization and Analytics,12,1
1,2,BC-WEBDEV,Full Stack Web Development,12,1
2,3,BC-UIUX,User Interface/User Experience,12,1
3,4,CIS-349,Introduction to Databases,5,0
4,5,CIS-405,Database Programming,5,0


In [23]:
course_data_df = course_data_df[['Name', 'Course_Code__c', 'Course_Name__c', 'Credit_Hours__c', 'Boot_Camp_Course__c']]
course_data_df.head()

,Name,Course_Code__c,Course_Name__c,Credit_Hours__c,Boot_Camp_Course__c
0,1,BC-DATAVIZ,Data Visualization and Analytics,12,1
1,2,BC-WEBDEV,Full Stack Web Development,12,1
2,3,BC-UIUX,User Interface/User Experience,12,1
3,4,CIS-349,Introduction to Databases,5,0
4,5,CIS-405,Database Programming,5,0


In [24]:
course_data_records = course_data_df.to_dict('records')
course_data_records

[{'Name': 1,
  'Course_Code__c': 'BC-DATAVIZ',
  'Course_Name__c': 'Data Visualization and Analytics',
  'Credit_Hours__c': 12,
  'Boot_Camp_Course__c': 1},
 {'Name': 2,
  'Course_Code__c': 'BC-WEBDEV',
  'Course_Name__c': 'Full Stack Web Development',
  'Credit_Hours__c': 12,
  'Boot_Camp_Course__c': 1},
 {'Name': 3,
  'Course_Code__c': 'BC-UIUX',
  'Course_Name__c': 'User Interface/User Experience',
  'Credit_Hours__c': 12,
  'Boot_Camp_Course__c': 1},
 {'Name': 4,
  'Course_Code__c': 'CIS-349',
  'Course_Name__c': 'Introduction to Databases',
  'Credit_Hours__c': 5,
  'Boot_Camp_Course__c': 0},
 {'Name': 5,
  'Course_Code__c': 'CIS-405',
  'Course_Name__c': 'Database Programming',
  'Credit_Hours__c': 5,
  'Boot_Camp_Course__c': 0},
 {'Name': 6,
  'Course_Code__c': 'CIS-438',
  'Course_Name__c': 'Database Administration',
  'Credit_Hours__c': 5,
  'Boot_Camp_Course__c': 0},
 {'Name': 7,
  'Course_Code__c': 'CIS-430',
  'Course_Name__c': 'Business Systems Programming I',
  'Credit_Ho

In [25]:
for rec in course_data_records:

    record = {
        'CourseCode__c': rec['Course_Code__c'],
        'CourseName__c': rec['Course_Name__c'],
        'CreditHours__c': rec['Credit_Hours__c'],
        'BootCampCourse__c': rec['Boot_Camp_Course__c'],
    }
    
    try:
        sf.Course__C.create(record)
    except Exception as e:
        print(e)

In [26]:
course_lookup_list = []

# The `Name` column in the primary key in Salesforce objects
data = sf.query_all_iter("SELECT CourseCode__c, Name FROM Course__c")
for row in data:
    rec = {
        'Course__c': row['Name'],
        'CourseCode__c': row['CourseCode__c']
    }
    course_lookup_list.append(rec)

In [27]:
course_lookup_df = pd.DataFrame(course_lookup_list)
course_lookup_df

,Course__c,CourseCode__c
0,a0M3h000002pGY3,BC-WEBDEV
1,a0M3h000002pGT7,CIS-435
2,a0M3h000002pGSn,CIS-349
3,a0M3h000002pGT2,CIS-430
4,a0M3h000002pGSi,BC-UIUX
5,a0M3h000002pGXy,BC-DATAVIZ
6,a0M3h000002pGXU,BC-UIUX
7,a0M3h000002pGXe,CIS-405
8,a0M3h000002pGSs,CIS-405
9,a0M3h000002pGYI,CIS-405


In [28]:
query = '''
    SELECT
        c.*
        ,co.CourseCode
    FROM
        class c
        INNER JOIN course co
        ON c.ID_Course = co.ID_Course
'''
class_data_df = pd.read_sql(query, conn)
class_data_df.head()

,ID_Class,ID_Course,Section,StartDate,EndDate,CourseCode
0,1,1,GWU-ARL-DATA-PT-09-0,2020-03-16,2020-03-14,BC-DATAVIZ
1,2,1,GWDC201805DATA3,2018-05-15,2018-11-08,BC-DATAVIZ
2,3,2,GWARL201905WEB3,2019-05-14,2019-11-07,BC-WEBDEV
3,4,3,GWARL201905UIUX3,2019-05-14,2019-11-07,BC-UIUX


In [36]:
 class_data_df.rename(columns={
    'Section':'Section__c',
    'StartDate':'Start_Date__c',
    'EndDate':'End_Date__c',
    'CourseCode':'Course_Code__c'
}, inplace=True)

class_data_df

,ID_Class,ID_Course,Section__c,Start_Date__c,End_Date__c,Course_Code__c
0,1,1,GWU-ARL-DATA-PT-09-0,2020-03-16,2020-03-14,BC-DATAVIZ
1,2,1,GWDC201805DATA3,2018-05-15,2018-11-08,BC-DATAVIZ
2,3,2,GWARL201905WEB3,2019-05-14,2019-11-07,BC-WEBDEV
3,4,3,GWARL201905UIUX3,2019-05-14,2019-11-07,BC-UIUX


In [37]:
class_data_df = pd.merge(class_data_df, course_lookup_df, how='left')
class_data_df.drop(columns = ['ID_Class','ID_Course','CourseName','CreditHours','BootCampCourse','Course_Code__c'], inplace=True)
class_data_df.head()

MergeError: No common columns to perform merge on. Merge options: left_on=None, right_on=None, left_index=False, right_index=False

In [38]:
class_data_df
class_data_df['Start_Date__c'] = pd.to_datetime(class_data_df['Start_Date__c']).dt.date
class_data_df['End_Date__c'] = pd.to_datetime(class_data_df['End_Date__c']).dt.date

class_data_df.head()

,ID_Class,ID_Course,Section__c,Start_Date__c,End_Date__c,Course_Code__c
0,1,1,GWU-ARL-DATA-PT-09-0,2020-03-16,2020-03-14,BC-DATAVIZ
1,2,1,GWDC201805DATA3,2018-05-15,2018-11-08,BC-DATAVIZ
2,3,2,GWARL201905WEB3,2019-05-14,2019-11-07,BC-WEBDEV
3,4,3,GWARL201905UIUX3,2019-05-14,2019-11-07,BC-UIUX


In [39]:
class_data_records = class_data_df.to_dict(orient='records')
class_data_records

[{'ID_Class': 1,
  'ID_Course': 1,
  'Section__c': 'GWU-ARL-DATA-PT-09-0',
  'Start_Date__c': datetime.date(2020, 3, 16),
  'End_Date__c': datetime.date(2020, 3, 14),
  'Course_Code__c': 'BC-DATAVIZ'},
 {'ID_Class': 2,
  'ID_Course': 1,
  'Section__c': 'GWDC201805DATA3',
  'Start_Date__c': datetime.date(2018, 5, 15),
  'End_Date__c': datetime.date(2018, 11, 8),
  'Course_Code__c': 'BC-DATAVIZ'},
 {'ID_Class': 3,
  'ID_Course': 2,
  'Section__c': 'GWARL201905WEB3',
  'Start_Date__c': datetime.date(2019, 5, 14),
  'End_Date__c': datetime.date(2019, 11, 7),
  'Course_Code__c': 'BC-WEBDEV'},
 {'ID_Class': 4,
  'ID_Course': 3,
  'Section__c': 'GWARL201905UIUX3',
  'Start_Date__c': datetime.date(2019, 5, 14),
  'End_Date__c': datetime.date(2019, 11, 7),
  'Course_Code__c': 'BC-UIUX'}]

In [40]:
for rec in class_data_records:
 
    record = {
        'ID_Class__c': rec['ID_Class__c'],
        'ID_Course__c': rec['ID_Course__c'],
        'Section__c': rec['Section__c'],
        'Start_Date__c': str(rec['Start_Date__c']),
        'End_Date__c': str(rec['End_Date__c']),
    }
    
    try:
        sf.Class__C.create(record)
    except Exception as e:
        print(e)

KeyError: 'ID_Class__c'